# Config


In [ ]:
import pandas as pd
from datetime import datetime
from realtime.vehicles import load_positions_metro
from static.stations import load_metro_stations

from config import EXTERNAL_DATA_DIR, PROCESSED_DATA_DIR


In [2]:
#Url Dinamic Data
VEHICLES_URL = "https://ctb-gtfs-rt.s3.eu-south-2.amazonaws.com/metro-bilbao-vehicle-positions.pb"

#Location static data
FOLDER = "Bizkaibus_GPKG"
gpkg_path = EXTERNAL_DATA_DIR / FOLDER / "Bizkaibus_GPKG.gpkg"


# Load data

## Get vehicles

In [3]:
df_metro = load_positions_metro(VEHICLES_URL)
df_metro

,vehicle_id,lat,lon,timestamp,mode
0,None,43.367039,-2.99949,2025-12-01 11:56:52,metro
1,None,43.261398,-2.92761,2025-12-01 11:56:52,metro
2,None,43.303379,-2.97437,2025-12-01 11:56:52,metro
3,None,43.327301,-2.99874,2025-12-01 11:56:52,metro
4,None,43.262589,-2.93308,2025-12-01 11:56:52,metro
5,None,43.243858,-2.89771,2025-12-01 11:56:52,metro
6,None,43.298851,-2.99268,2025-12-01 11:56:52,metro
7,None,43.274948,-2.95874,2025-12-01 11:56:52,metro
8,None,43.350449,-3.00935,2025-12-01 11:56:52,metro
9,None,43.290981,-2.96992,2025-12-01 11:56:52,metro


## Get stations

In [ ]:
stops = gpd.read_file(gpkg_path, layer="Geralekuak_Paradas")

## Get lines

In [25]:
# List all layer names in the GeoPackage
layers = gpd.list_layers(gpkg_path)["name"]

# Exclude the "Geralekuak_Paradas" layer
layers_to_read = [lyr for lyr in layers if lyr != "Geralekuak_Paradas"]

gdfs = []
for lyr in layers_to_read:
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="Non-conformant content for record",
            category=RuntimeWarning,
            module="pyogrio",
        )
        gdf = gpd.read_file(gpkg_path, layer=lyr)

    gdf["layer_name"] = lyr
    gdfs.append(gdf)


# Combine into one GeoDataFrame
lines = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Data management (Code)

## Clean Stations & Stops

In [32]:
stops.describe()

,CodigoReducidoParada,CodigoProvincia,DescripcionProvincia,CodigoMunicipio,DescripcionMunicipio,Denominacion,CodificacionRuta,geometry
count,2353,2353,2353,2353,2353,2353,2353,2353
unique,2353,4,4,113,121,1392,1116,2353
top,4026,48,BIZKAIA,020,BILBAO,Elexalde,A3523_Bilbao-Hospital Galdakao Ospitalea-Gerni...,POINT (499127 4762865)
freq,1,2251,2251,117,117,17,59,1


# Plots

In [3]:
# Plots and visualizations

# Save results

In [ ]:
#Save results and figures
FOLDER = FOLDER.split("_GPKG")[0]
(PROCESSED_DATA_DIR / FOLDER).mkdir(parents=True, exist_ok=True)
lines.to_file(PROCESSED_DATA_DIR / FOLDER / "bizkaibus_lines.gpkg", layer="lines")
stops.to_file(PROCESSED_DATA_DIR / FOLDER / "bizkaibus_stops.gpkg", layer="stops")